In [ ]:
#import and lode modules
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
Root="/content/drive/MyDrive/NewDataset3/Brain Tumour1"
number_of_images={}
for dir in os.listdir(Root):
  number_of_images[dir]=len(os.listdir(os.path.join(Root,dir)))


In [ ]:
number_of_images.items()

dict_items([('Non tumour', 2075), ('Tumour', 1687)])

In [ ]:
#Split the data -> 70% for train, 15% for validation, 15% for testing

def dataFolder(p,split):

  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

    for dir in os.listdir(Root):
      os.makedirs("./"+p+"/"+dir)

      for img in np.random.choice(a=os.listdir(os.path.join(Root,dir)),
                                  size=(math.floor(split*number_of_images[dir])-5),
                                  replace=False):

        O=os.path.join(Root,dir,img)
        D=os.path.join("./"+p,dir)
        shutil.copy(O,D)
        #os.remove(O)

  else:
    print( f"{p}folder exists")


In [ ]:
dataFolder("train",0.7)

In [ ]:
dataFolder("validation",0.15)

In [ ]:
dataFolder("test",0.15)

In [ ]:
#Model build

from keras.layers import Conv2D, MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras


In [ ]:
import keras
print(keras.__version__)

2.12.0

In [ ]:

#CNN model

from keras.models import Sequential  # Make sure to import Sequential with an uppercase "S"
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(Conv2D(filters=36, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(GlobalAvgPool2D())  # Add Global Average Pooling layer

model.add(Dropout(rate=0.25))

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 36)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 109, 109, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0         
 2D)                                                             
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 128)               0         
...
Total params: 116,965
Trainable params: 116,965
Non-trainable params: 0
_________________________________________________________________
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
#use data generator

def preImage1(path):
  """
  input:path
  output:pre proccessed images
  """
  image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255,horizontal_flip=True)  #data augmentation

  image=image_data.flow_from_directory(directory=path,target_size=(244,244),batch_size=32,class_mode='binary')

  return image


In [ ]:
from tensorflow.python.util.compat import path_to_str
path="/content/train"

train_data=preImage1(path)

In [ ]:
#use data generator

def preImage2(path):
  """
  input:path
  output:pre proccessed images
  """
  image_data=ImageDataGenerator(rescale=1/255)#no data augmentation in testing

  image=image_data.flow_from_directory(directory=path,target_size=(244,244),batch_size=32,class_mode='binary')

  return image

In [ ]:
path="/content/test"

test_data=preImage2(path)

Found 554 images belonging to 2 classes.

In [ ]:
path="/content/validation"

val_data=preImage2(path)

Found 554 images belonging to 2 classes.

In [ ]:
#early stopping and model check point

from keras.callbacks import ModelCheckpoint,EarlyStopping

#early stopping

es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=12,verbose=1,mode='auto')

#model checkpoint


mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",verbose=1,save_best=True,mode='auto')

cd=[es,mc]

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Model training

hs=model.fit_generator(generator=train_data,
                       steps_per_epoch=8,
                       epochs=30,
                       verbose=1,
                       validation_data=val_data,
                       validation_steps=16,
                       callbacks=cd)


In [ ]:
#Model graphical interpretation

h=hs.history
h.keys()


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h['accuracy'])

plt.plot(h['val_accuracy'],c='red')

plt.title('Accuracy vs Validation Accuracy')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h['loss'])

plt.plot(h['val_loss'],c='red')

plt.title('Loss vs Validation Loss')
plt.show()